在人民币走向国际化的战略中，实现浮动汇率制是关键一步也是必要的一步。

过分依赖于外汇干预实施外汇管控不仅会对外汇储备造成巨大压力、非冲销式干预造成货币超发，也不利于汇率发挥自动调节国际收支的作用。

参考《人民币从市场化走向国际化》，本文将通过实证，分析不同汇率制下的经济体面对外部冲击时，国内主要经济指标的不同反应。

# 实证

    1、数据选取，来源Bloomberg

选取26个国家：玻利维亚、印度、马来西亚、墨西哥、新西兰、挪威、秘鲁、波兰、罗马尼亚、俄罗斯、韩国、泰国、土耳其、澳大利亚、巴西、加拿大、智利、哥伦比亚、捷克、匈牙利、印度尼西亚、以色列、菲律宾、南非、斯里兰卡、瑞典

经济指标：VIX指数、兑美元名义汇率、外汇储备变化净值、CPI、PPI、股票指数、对美联储短期利率差

    2、实证设计

2.1 VAR建模

【1】参考：https://blog.csdn.net/mooncrystal123/article/details/86736397?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522162367984316780274168623%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=162367984316780274168623&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~times_rank-1-86736397.first_rank_v2_pc_rank_v29&utm_term=python+向量自回归&spm=1018.2226.3001.4187

In [1]:
import statsmodels as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

1）首先画出两个时间序列的序列相关图：

In [ ]:
def serial_correlation_plot(ts1 , ts2):
    ts1 = (ts1 - np.mean(ts1))/np.std(ts1)
    ts2 = (ts2 - np.mean(ts2))/np.std(ts2)
    plt.plot(ts1 , col = 'r' , label = 'VIX')
    plt.plot(ts2 , col = 'b' , label = str(ts2) + 'foreign reserve')
    plt.title('Correlation: ' + str(ts2) , fontsize = 20)
    plt.show()

var模型要求两个时间序列同阶协整，因此进行平稳性检验

2）ADF单位根检验：

In [ ]:
def adf_test(data):
    
    adfresult = sm.tsa.stattools.adfuller(data)
    
    print('p value : ' + float(adfresult[1])) 
    print('lag : ' + adfresult[2])
    print(adfresult)

ps：由于ADF检验是对于原序列的差分序列进行单整检验，改变了时间序列的经济含义，所以我们仍希望使用原序列进行建模。

对于同阶单整的两个序列可以检验是否协整。

3）协整检验：

ps：就像你遛狗一样，狗虽然到处乱跑，但由于被你用绳子牵着，长期来看与你的方向是一致的（协整关系）

In [ ]:
def cointegration_test(ts1 , ts2):
    result = sm.tsa.stattools.coint(ts1 , ts2)
    print(result)

4）var建模：

In [ ]:
def var_model(data):
    
    #data需是一个np.arrray
    
    orgMod = sm.tsa.VARMAX(data , order=(varLagNum,0) , trend='nc' , exog=None)

    fitMod = orgMod.fit(maxiter=1000,disp=False)

    print(fitMod.summary())

    resid = fitMod.resid #残差序列
    result = {'fitMod':fitMod,'resid':resid}

选取向量自回归模型(VAR)

首先对VIX与外汇干预指标进行建模，以区分外汇干预国与汇率浮动国

# 参考文献

【1】《人民币从国际化走向市场化》

【2】Blanchard O, Adler G, de Carvalho Filho I. Can foreign exchange intervention stem exchange rate pressures from global capital flow shocks?[R]. National Bureau of Economic Research, 2015.

【3】Gourinchas P O, Rey H. From world banker to world venture capitalist: US external adjustment and the exorbitant privilege[M]//G7 current account imbalances: sustainability and adjustment. University of Chicago Press, 2007: 11-66.

【4】Adler G, Lisack N, Mano R. Unveiling the effects of foreign exchange intervention: a panel approach[M]. International Monetary Fund, 2015.